### 用于特征提取和图像匹配变换

In [1]:
import cv2
import numpy as np
import PIL
from PIL import Image

In [2]:
# 1. 特征提取
def extract_features(image):
    feature_extractor = cv2.SIFT_create(nfeatures=10000,nOctaveLayers=4,contrastThreshold=0.02,edgeThreshold=10,sigma=1.0)
    keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
    return keypoints, descriptors

# def extract_features_orb(image):
#     orb = cv2.ORB_create(nfeatures=3000)
#     keypoints, descriptors = orb.detectAndCompute(image, None)
#     if descriptors is not None:
#         descriptors = descriptors.astype(np.float32)
#     else:
#         descriptors = np.array([])  # 或者设置一个默认的空描述符
#     return keypoints, descriptors

# 2. 特征匹配
def match_features(descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

# 3. 变换估计
def estimate_transform(keypoints1, keypoints2, good_matches):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    # 使用RANSAC算法估计变换矩阵
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return M

#4. 变换应用
def apply_transform(image, M):
    # 应用估计的变换矩阵到图像
    transformed_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))
    return transformed_image

In [3]:
# 一次配准
image1 = cv2.imread('origin_images/2.png', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('origin_images/222.png', cv2.IMREAD_GRAYSCALE)

# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)

# 2. 特征匹配
good_matches = match_features(descriptors1, descriptors2)

# 3. 变换估计
M = estimate_transform(keypoints1, keypoints2, good_matches)

# 4. 变换应用
transformed_image = apply_transform(image1, M)
cv2.imwrite('transformed2to222-0.5.png', transformed_image)

True

In [4]:
# def pyramid_match_features(image1, image2, levels=3):
#     # 创建图像金字塔
#     pyramid1 = [image1]
#     pyramid2 = [image2]
#     for _ in range(levels - 1):
#         image1 = cv2.pyrDown(image1)
#         image2 = cv2.pyrDown(image2)
#         pyramid1.append(image1)
#         pyramid2.append(image2)
#     # 存储金字塔中每一级的变换矩阵
#     transforms = []
#     for level in range(levels):
#         # 提取特征
#         keypoints1, descriptors1 = extract_features(pyramid1[level])
#         keypoints2, descriptors2 = extract_features(pyramid2[level])
#         # 特征匹配
#         good_matches = match_features(descriptors1, descriptors2)
#         # 变换估计
#         M = estimate_transform(keypoints1, keypoints2, good_matches)
#         # 保存变换矩阵
#         transforms.append(M)
#         # 向上传递变换
#         if level > 0:
#             M = np.vstack((M, [0, 0, 1]))  # 添加齐次坐标
#             M_inv = np.linalg.inv(M)
#             pyramid2[level - 1] = apply_transform(pyramid2[level - 1], M_inv)
#     # 最终变换矩阵
#     final_M = transforms[-1]
#     return final_M

In [5]:
# M_pyra = pyramid_match_features(image1,image2)
# M_pyra = M_pyra.astype(np.float32)  # 将变换矩阵转换为浮点数类型
# # if M_pyra.shape != (3, 3):
#     # raise ValueError("变换矩阵的维度不正确")
# # 添加齐次坐标
# M_pyra = np.vstack((M_pyra, [0, 0, 1]))

# # 应用变换
# transformed_image = cv2.warpPerspective(image1, M_pyra, (image1.shape[1], image1.shape[0]))
# cv2.imwrite('transformed2to222_pyra.png', transformed_image)

In [6]:
# # 二次配准
# image11 = transformed_image
# # 1. 特征提取
# keypoints11, descriptors11 = extract_features(image11)
# # 2. 特征匹配
# good_matches2 =match_features(descriptors11, descriptors2)
# # 3. 变换估计
# M = estimate_transform(keypoints11, keypoints2, good_matches2)
# # 4. 变换应用
# transformed_image = apply_transform(image11, M)
# cv2.imwrite('2transformed2to222.png', transformed_image)

### 后面是SIFT和BFMatch合理性判断

In [7]:
#一次配准
# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)

img3 = cv2.drawKeypoints(image1,keypoints1,image1,color=(255,0,255))
img4 = cv2.drawKeypoints(image2,keypoints2,image2,color=(255,0,255))
hmerge = np.hstack((img3, img4))
# pillow_image = Image.fromarray(cv2.cvtColor(hmerge, cv2.COLOR_BGR2RGB))
# pillow_image.show()

In [ ]:
# BFMatcher解决匹配
#可能还得试试其他的匹配方法和
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1,descriptors2, k=2)
#print(matches)
#print(len(matches))
good = []
for m,n in matches:
    if m.distance < 0.5*n.distance:
        good.append([m])
img5 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,matches,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

pillow_image = Image.fromarray(cv2.cvtColor(img5, cv2.COLOR_BGR2RGB))
pillow_image.show()

In [ ]:
img6 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,good,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

pillow_image = Image.fromarray(cv2.cvtColor(img6, cv2.COLOR_BGR2RGB))
pillow_image.show()